# <font color = 'orange'> Data Ingestion

---

In [1]:
import os

In [2]:
%pwd

'e:\\PW Skills - Data Science\\Projects\\09_Chest_Cancer_Classification\\notebook'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\PW Skills - Data Science\\Projects\\09_Chest_Cancer_Classification'

---

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:              # Values for this are present in the 'config.yaml' file
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from chest_cancer_classifier.constants import *
from chest_cancer_classifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:          # used to set the "values present in yaml" file to "DataIngestionConfig variables" 
        ''' returns own custom data type '''
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [7]:
import os
import zipfile
import gdown
from chest_cancer_classifier import logger


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
            
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
        try: 
            os.makedirs(self.config.root_dir, exist_ok=True)
            
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")            
            
            prefix = 'https://drive.google.com/uc?/export=download&id='
            file_id = dataset_url.split("/")[-2]
            download_url = prefix + file_id
            gdown.download(download_url, zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f'Extracted data from zip file')

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-21 15:36:05,098: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-21 15:36:05,106: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-21 15:36:05,106: INFO: common: created directory at: artifacts]
[2024-04-21 15:36:05,106: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-21 15:36:05,115: INFO: 4111220501: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=e8302157-d09c-4b53-b091-cf48ec94f0fa
To: e:\PW Skills - Data Science\Projects\09_Chest_Cancer_Classification\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:23<00:00, 2.06MB/s]

[2024-04-21 15:36:31,570: INFO: 4111220501: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


[2024-04-21 15:36:32,722: INFO: 4111220501: Extracted data from zip file]


---